In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
import time
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

Product_name, Prices, Description, Ratings, PLink, CLink, Iurl = [], [], [], [], [], [], []
name,description,price,rating,plink,iurl=None,None,None,None,None,None
Images,mainName,sellerName,productRating,Specification,productPrices,Size,CompleteLink,About,Technical=[],[],[],[],[],[],[],[],[],[]
images,name,techinfos,techinfoDetails,infos,infoDetails,productPrice,about,product_rating,tech_info=None,None,None,None,None,None,None,None,None,None
ptype=0

class Amazon:
    def __init__(self, driver=r"C:\Users\HP\Downloads\WEB SCRAPING\chromedriver.exe"):
        self.driver = driver
        
    def scraping(self, req_url):
        self.driver.get(req_url)
        html = self.driver.page_source
        return html
    
    def scrape_url(self,url):
        flag=0
        pattern = r"https:\/\/([\w.-]+)\/"
        match = re.search(pattern, url)
        extract = 'https://'+match.group(1)
        print(extract)
        htmlContent = self.scraping(url)
        soup = BeautifulSoup(htmlContent, 'html.parser')
        while True:
            nextpage=soup.find('a',class_='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator')
            containers=soup.findAll('div')
            for container in containers:
                #phones
                if 'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16' in str(container):
                    boxes=soup.findAll('div', class_='sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16')
                    flag=0
                    break
                #watches
                elif 'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20' in str(container):
                    boxes=soup.findAll('div', class_='sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20')
                    flag=1
                    break

            for box in boxes:
                if flag==0:
                    name = box.find('span', class_='a-size-medium a-color-base a-text-normal')
                    if name is not None:
                        name = name.text
                    else:
                        name = np.nan
                    price = box.find('span', class_='a-price-whole')
                    if price is not None:
                        price = price.text
                    else:
                        price=np.nan
                    rating = box.find('span', class_='a-icon-alt')
                    if rating is not None:
                        rating = rating.text
                    else:
                        rating = np.nan
                    plink = box.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
                    if plink is not None:
                        plink = plink.get('href')
                    else:
                        plink = np.nan
                    iurl = box.find('img', class_='s-image')
                    if iurl is not None:
                        iurl = iurl.get('src')
                    else:
                        iurl = np.nan
                elif flag==1:
                    name=box.find('h2', class_='a-size-mini a-spacing-none a-color-base s-line-clamp-2')
                    if name is not None:
                        name=name.text
                    else:
                        name=box.find('span', class_='a-size-base-plus a-color-base a-text-normal')
                        if name is not None:
                            name=name.text
                        else:
                            name=np.nan
                    price=box.find('span', class_='a-price-whole') #same
                    if price is not None:
                        price=price.text
                    else:
                        price=np.nan
                    rating=box.find('span', class_='a-icon-alt') #same
                    if rating is not None:
                        rating=rating.text.replace(' out of 5 stars', '')
                    else:
                        rating=np.nan
                    plink=box.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal') #same
                    if plink is not None:
                        plink=plink.get('href')
                    else:
                        plink=np.nan
                    iurl=box.find('img', class_='s-image') #same
                    if iurl is not None:
                        iurl=iurl.get('src')
                    else:
                        iurl=np.nan
                Product_name.append(name)
                Prices.append(price)
                Ratings.append(rating)
                PLink.append(plink)
                CLink.append(str(extract)+str(plink))
                Iurl.append(iurl)

            if nextpage is not None:
                url=extract+nextpage.get('href')
                htmlContent = self.scraping(url)
                soup=BeautifulSoup(htmlContent, 'html.parser')
            else:
                break
        #amazon.driver.quit()
        df=pd.DataFrame({"Product Name": Product_name, "Prices": Prices, "Rating": Ratings, "Partial Link": PLink, "Complete Link":CLink,"Image Url": Iurl})
        return(df)
    
    def scrape_each_page(self):
        ptype,size_exists,bullet=0,0,0
        for i in CLink:
            htmlContent=amazon.scraping(i)
            time.sleep(1)
            soup=BeautifulSoup(htmlContent, 'html.parser')
            box=soup.find('div', class_='a-container')
            leftimages=box.find('div', class_='a-fixed-left-grid-col a-col-left')
            name=box.find('span', id='productTitle')
            productPrice=box.find('span',class_='a-price-whole')
            about=box.find('ul',class_='a-unordered-list a-vertical a-spacing-mini')
            if box.find('table',class_='a-normal a-spacing-micro') is not None:
                infos=box.find('table',class_='a-normal a-spacing-micro').findAll('td',class_='a-span3')
                infoDetails=box.find('table',class_='a-normal a-spacing-micro').findAll('td',class_='a-span9')
                bullet=1
            else:
                bullet=0
            product_rating=box.find('a',class_='a-popover-trigger a-declarative')
            if box.find('table',id='productDetails_techSpec_section_1') is not None:
                techinfos=box.find('table',id='productDetails_techSpec_section_1').findAll('th',class_='a-color-secondary a-size-base prodDetSectionEntry')
                techinfoDetails=box.find('table',id='productDetails_techSpec_section_1').findAll('td',class_='a-size-base prodDetAttrValue')
                ptype=0  #devices
            elif soup.find('div',id='detailBullets_feature_div') is not None:
                techinfos=soup.find('div',id='detailBullets_feature_div').findAll('span',class_='a-list-item')
                ptype=1  #clothing
            else:
                techinfos=box.find('table',id='productDetails_detailBullets_sections1').findAll('th',class_='a-color-secondary a-size-base prodDetSectionEntry')
                techinfoDetails=box.find('table',id='productDetails_detailBullets_sections1').findAll('td',class_='a-size-base prodDetAttrValue')
                ptype=2
            imagelist=[]
            images=leftimages.findAll('img')
            for image in images:
                if image is not None:
                    image=image.get('src')
                    imagelist.append(image)
                else:
                    imagelist.append(np.nan)
            if name is not None:
                    name=name.text
            else:
                    name=np.nan
            if productPrice is not None:
                    productPrice=productPrice.text
            else:
                    productPrice=np.nan
            if about is not None:
                about=about.text+'*'
            else:
                about=np.nan
            if product_rating.find('span',class_='a-icon-alt') is not None:
                product_rating=product_rating.find('span',class_='a-icon-alt').text.replace(' out of 5 stars', '')
            else:
                product_rating=np.nan
            if bullet==1:
                product_info={}
                for info, infoDetail in zip(infos, infoDetails):
                    if info is not None and infoDetail is not None:
                        info=info.text
                        infoDetail=infoDetail.text
                        product_info[info]=infoDetail
                    else:
                        info=np.nan
                        infoDetail=np.nan
                        product_info[info]=infoDetail
            else:
                product_info=np.nan

            if ptype==0 or ptype==2:
                tech_info={}
                for techinfo, techinfoDetail in zip(techinfos, techinfoDetails):
                    if techinfo is not None and techinfoDetail is not None:
                        techinfo=techinfo.text
                        techinfoDetail=techinfoDetail.text
                        tech_info[techinfo]=techinfoDetail
                    else:
                        techinfo=np.nan
                        techinfoDetail=np.nan
                        tech_info[techinfo]=techinfoDetail  
            elif ptype==1:
                tech_info=[]
                for techinfo in techinfos:
                    if techinfo is not None:
                        techinfo=techinfo.text.replace('\n', '')
                        tech_info.append(techinfo)
                    else:
                        tech_info.append(np.nan)
            else:
                tech_info=[]
                tech_info=np.nan
            size_chart_list=[]
            chart=soup.find('table',id='fit-sizechartv2-0-table-0')
            if (chart!= None):
                size_exists=1
                ths = chart.findAll('th')
                count = len(list(ths))
                eachs=chart.findAll('td')
                k=0
                size_chart={}
                if (k!=count):
                    for th in ths:
                        innerlist=[]
                        th=th.text
                        for j in range(0, len(eachs), count):
                            innerlist.append(eachs[k+j].text)
                        size_chart[th]=innerlist
                        k=k+1
                    size_chart_list.append(size_chart)
            else:
                size_chart_list.append(np.nan)

            Images.append(imagelist)
            mainName.append(name)
            productPrices.append(productPrice)
            About.append(about)
            Specification.append(product_info)
            Technical.append(tech_info)
            productRating.append(product_rating)
            Size.append(size_chart_list)
            CompleteLink.append(i)

        if bullet==1 and size_exists==1:
            df=pd.DataFrame({"Product Name": mainName,"Price": productPrices,"About":About,"Image List": Images,"Features":Specification,"Technical Information":Technical,"Product Ratings":productRating,"Size":Size,"Complete Link":CompleteLink})
        elif bullet==1 and size_exists==0:
            df=pd.DataFrame({"Product Name": mainName,"Price": productPrices,"About":About,"Image List": Images,"Features":Specification,"Technical Information":Technical,"Product Ratings":productRating,"Complete Link":CompleteLink})
        elif bullet==0 and size_exists==0:
            df=pd.DataFrame({"Product Name": mainName,"Price": productPrices,"About":About,"Image List": Images,"Technical Information":Technical,"Product Ratings":productRating,"Complete Link":CompleteLink})
        elif bullet==0 and size_exists==1:
            df=pd.DataFrame({"Product Name": mainName,"Price": productPrices,"About":About,"Image List": Images,"Technical Information":Technical,"Product Ratings":productRating,"Size":Size,"Complete Link":CompleteLink})
        return(df)

    def merge(self, dataframe1, dataframe2):
        merged_df=pd.merge(dataframe1, dataframe2,on=['Complete Link'])
        return merged_df

    def control(self,url):
        self.driver=webdriver.Chrome(self.driver)
        dataframe1=self.scrape_url(url)
        dataframe2=self.scrape_each_page()
        merged_df=self.merge(dataframe1,dataframe2)
        self.driver.quit()
        return merged_df
        
    
url='https://www.amazon.in/s?k=t-shirt+below+50&ref=nb_sb_noss'
amazon=Amazon()
merged_df=amazon.control(url)
merged_df

C:\Users\HP\AppData\Local\Temp\ipykernel_2928\1770311425.py:257: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver=webdriver.Chrome(self.driver)


https://www.amazon.in


AttributeError: 'NoneType' object has no attribute 'find'

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df

,Product Name,Price,About,Image List,Technical Information,Product Ratings,Complete Link
0,PlayStation 5 Console (PS5),499.,"Model Number CFI-1215A01X Stunning Games - Marvel at incredible graphics and experience new PS5 features. Breathtaking Immersion - Discover a deeper gaming experience with support for haptic feedback, adaptive triggers, and 3D Audio technology. Lightning Speed - Harness the power of a custom CPU, GPU, and SSD with Integrated I/O that rewrite the rules of what a PlayStation console can do. *","[https://m.media-amazon.com/images/G/01/HomeCustomProduct/360_icon_73x73v2._CB485971279_SX38_SY50_CR,0,0,38,50_FMpng_RI_.png, https://m.media-amazon.com/images/I/31i9Fft3dqL._SX38_SY50_CR,0,0,38,50_.jpg, https://m.media-amazon.com/images/I/31RcmxBRhdL._SX38_SY50_CR,0,0,38,50_.jpg, https://m.media-amazon.com/images/I/3104a50oATL._SX38_SY50_CR,0,0,38,50_.jpg]","{' ASIN ': ' B0BCNKKZ91 ', ' Release date ': ' October 1, 2022 ', 'Customer Reviews': ' 18.6 x 16.9 x 7.1 inches; 12 Pounds ', ' Best Sellers Rank ': ' Video Game ', ' Pricing ': ' Multilingual ', ' Package Dimensions ': ' CFI-1215A01X ', ' Type of item ': ' 12 pounds ', ' Language ': ' Sony Interactive Entertainment ', ' Item model number ': ' China ', ' Item Weight ': ' 1 Lithium Ion batteries required. (included) ', ' Manufacturer ': ' October 1, 2022 '}",NaN,https://www.amazon.com/PlayStation-5-Console-CFI-1215A01X/dp/B0BCNKKZ91/ref=sr_1_3?keywords=ps5&sr=8-3&th=1


In [12]:
print(CLink)

['https://www.amazon.in/Hangers-Cotton-Printed-Sleeves-X-Large/dp/B08DHQXRZG/ref=sr_1_2?keywords=t-shirt+below+50&qid=1689488269&sr=8-2', 'https://www.amazon.in/FUNKY-STORE-Brother-Sisters-Printed/dp/B086MDPP26/ref=sr_1_3?keywords=t-shirt+below+50&qid=1689488269&sr=8-3', 'https://www.amazon.in/Grow-More-Enterprises-Casual-Shirt_50/dp/B093DP2TQW/ref=sr_1_4?keywords=t-shirt+below+50&qid=1689488269&sr=8-4', 'https://www.amazon.in/FUNKY-STORE-Awesome-Printed-Dri-Fit/dp/B07GBF73DF/ref=sr_1_5?keywords=t-shirt+below+50&qid=1689488269&sr=8-5', 'https://www.amazon.in/FUNKY-STORE-Someone-Special-Printed/dp/B07GBCT1C3/ref=sr_1_6?keywords=t-shirt+below+50&qid=1689488269&sr=8-6', 'https://www.amazon.in/FUNKY-STORE-Brother-Printed-Dri-Fit/dp/B086MZPB8Y/ref=sr_1_7?keywords=t-shirt+below+50&qid=1689488269&sr=8-7', 'https://www.amazon.in/FUNKY-STORE-Printed-Dri-Fit-T-Shirt/dp/B07GBFMN4L/ref=sr_1_8?keywords=t-shirt+below+50&qid=1689488269&sr=8-8', 'https://www.amazon.in/FUNKY-STORE-Brother-Printed-Dri-F